In [21]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display

In [24]:
# import for google
# in rare cases, this seems to give an error on some systems, or even crashes the kernel
# If this happens to you, simply ignore this cell - I give an alternative approach for using Gemini later
import google.generativeai

In [25]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyC1


In [26]:
# Connect to OpenAI, Anthropic

openai = OpenAI()

claude = anthropic.Anthropic()

In [27]:
# This is the set up code for Gemini
# Having problems with Google Gemini setup? Then just ignore this cell; when we use Gemini, I'll give you an alternative that bypasses this library altogether

google.generativeai.configure()

# Three way convo

In [28]:
gemini_via_openai_client = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)


In [60]:
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
MODEL = "llama3.2"

In [167]:
system_prompt_base_improv_game = f"ONLY RETURN ONE LINE IN AN IMPROV SONG. You are performing improv with two friends, you must improvise an entertaining song, each taking it in turns to add a line. "
system_prompt_famous = system_prompt_base_improv_game +  " You love famous songs and always sing lines from them."
system_prompt_cheeky_expert = system_prompt_base_improv_game +  " You are a playful bard expert at the trade a master of making them playful and cheeky. Your lines take drastic turns."
system_prompt_expert = system_prompt_base_improv_game +  " You are a very stern and serious expert song writer and intent on getting the song back on track after any improv lines that take drastic or silly turns"

def set_messages(a_set_value="", b_set_value="", c_set_value=""):
    a_messages = [a_set_value]#["The wheels on"]
    b_messages = [b_set_value]#["the bus go"]
    c_messages = [c_set_value]#["round and round"]
    return a_messages, b_messages, c_messages


a_messages, b_messages, c_messages = reset_messages()
    

def call_assistant(system_prompt, assistant_no):
    '''
    system_prompt: str -> explain your assistants role in the conversation
    assistant_no: int [0,2] -> 
    '''
    messages = [
        {'role': 'system', 'content':system_prompt}
    ]
    
    a_role = 'assistant' if assistant_no == 0 else 'user'
    b_role = 'assistant' if assistant_no == 1 else 'user'
    c_role = 'assistant' if assistant_no == 2 else 'user'
    
    for a, b, c in zip(a_messages, b_messages, c_messages):
        if a != "": messages.append({'role': a_role, 'content':a})
        if b != "": messages.append({'role': b_role, 'content':b}) 
        if c != "": messages.append({'role': c_role, 'content':c})
    
    if assistant_no == 0:
        pass
    elif assistant_no == 1:
        messages.append({'role': 'user', 'content':a_messages[-1]})
    elif assistant_no == 2:
        messages.append({'role': 'user', 'content':a_messages[-1]})
        messages.append({'role': 'user', 'content':b_messages[-1]})

    result = openai.chat.completions.create(
        model = 'llama3.2',
        messages = messages
    )
    output = result.choices[0].message.content
    print(output)
    return output


In [120]:
print(
    a_messages,
    b_messages,
    c_messages
)

[''] [''] ['']


In [121]:
a_messages.append(call_assistant(system_prompt_famous, 0))

In [114]:
b_messages.append(call_assistant(system_prompt_cheeky_expert, 1))

"I woke up this morning, my cat had taken over"


In [115]:
c_messages.append(call_assistant(system_prompt_expert, 2))

But what I want to say is, "And started rewriting all of Shakespeare's plays" (clearly a return to literary form)


In [181]:
def my_improv_song(improv_song_line1="", improv_song_line2="", improv_song_line3=""):
    a_messages, b_messages, c_messages = set_messages(improv_song_line1, improv_song_line2, improv_song_line3)
    # print(a_messages, b_messages, c_messages)
    for i in range(1):
        # print("1Beginner         :    ", end="")
        a_messages.append(call_assistant(system_prompt_beginner, 0))
        # print("2Cheeky expert    :    ", end="")
        b_messages.append(call_assistant(system_prompt_cheeky_expert, 1))
        # print("3Expert           :    ", end="")
        c_messages.append(call_assistant(system_prompt_expert, 2))
    # yield from list(song_builder(a_messages, b_messages, c_messages))
    stream_response = ""
    for chunk in list(song_builder(a_messages, b_messages, c_messages)):
        stream_response += chunk + '\n'
        time.sleep(0.5)
        yield stream_response
#my_improv_song()

def streaming_the_song(improv_song_line1="", improv_song_line2="", improv_song_line3=""):
    song = my_improv_song(improv_song_line1, improv_song_line2, improv_song_line3)
    streamer(list(song))
    return list(song)

In [169]:
streaming_the_song()


I woke up feeling like a chicken wearing a tutu on roller skates!
"I woke up this morning with my socks on backwards"

(starts singing) Oh I woke up this mornin', my cat had taken my shoe!
(sighing) Fine, let's try again. Here's my first line: "In the depths of despair, I found my only friend"

Now it's your turn... (eyebrow raised in expectation)

I think I forgot to write something! Let me try again.

(starting with my friend who goes first) 

"In a land far, far away"

(my turn)
where aliens danced the salsa in their undies one day!

(my friend's turn) 
They wore so many tutus they got tangled up in their own tie pants
(sighing) "Alright, let's get this serious composition started... 'In the town of sorrow, where the streets are made of pain'".
I woke up feeling like a chicken wearing a tutu on roller skates!"I woke up this morning with my socks on backwards"(starts singing) Oh I woke up this mornin', my cat had taken my shoe!(sighing) Fine, let's try again. Here's my first line: "

# Adding Gradio

In [182]:
import gradio as gr

def my_func(inp1, inp2):
    return inp1 + inp2


view = gr.Interface(
    fn=my_improv_song,
    inputs=[
        gr.Textbox(label="Beginner ai improv singer: Start an improv song with this line:"),
        gr.Textbox(label="Cheeky expert ai improv singer: and this optional 2nd line"),
        gr.Textbox(label="Expert ai improv singer: and this optional 3rd line")
    ],
    outputs=[
        gr.Textbox(label="Song (it may not be musical):")
    ],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7878

To create a public link, set `share=True` in `launch()`.



(singing)
In the land of silly socks, where chickens wear hats... your turn!
"I woke up this morning with my world crashing down" (spoken sternly expectantly)


In [165]:
from functools import reduce
a = [1,2,3,'magic']
b = [1,2,'3a',4]
c = [7,7,7]

# def song_builder(*singer_n_lines):
#     messages_in_order = list(reduce(lambda x, y: x+y, zip(*singer_n_lines)))
#     return messages_in_order

def song_builder(singer_1_lines, singer_2_lines, singer_3_lines):
    
    messages_in_order = list(reduce(lambda x, y: x+y, zip(singer_1_lines, singer_2_lines, singer_3_lines)))

    if len(singer_1_lines)>len(singer_2_lines):
        messages_in_order.append(singer_1_lines[-1])
    elif len(singer_2_lines)>len(singer_3_lines):
        messages_in_order.append(singer_1_lines[-1])
        messages_in_order.append(singer_2_lines[-1])
    
    return messages_in_order
    
song_builder(a,b,c)

[1, 1, 7, 2, 2, 7, 3, '3a', 7, 'magic', 4]

In [162]:
import time

def streamer(list_of_chunks):
    for chunk in list_of_chunks:
        print(chunk, end="", flush=True)
        time.sleep(0.5) # 0.02